In [1]:
import numpy as np
from mesh.mesh_read_plot3D import StructuredMeshInitialization2D
from mesh.mesh import MeshGeoCalculator2D
import boundary.ghost_cell as gc
import type_transform as tf
import config

# 读取并预处理网格

In [2]:
mesh_read = StructuredMeshInitialization2D()
mesh_read.load_file("airfoil0012extend.grd", "airfoil0012extend.inp", 0.001)
mesh_read.merge_blocks_2D()
mesh_read.interface_transform_cal()
mesh_read.mesh_reader.info()
mesh_read.bc_reader.info()
mesh_read.print_block_info()

[Block 0] shape: (259, 49, 1)
  x range: -14.898221 ~ 15.908266
  y range: -15.695282 ~ 15.697121
  z range: 0.000000 ~ 0.000000
[Block 1] shape: (10, 49, 1)
  x range: 1.000000 ~ 15.908266
  y range: -0.163128 ~ 0.215341
  z range: 0.000000 ~ 0.000000
Detected 2D boundary condition file.
[Block 0] A shape: (259, 49, 1)
  - Source: (259, 130, 1, 1), Type: 2
  - Source: (259, 259, 1, 49), Type: -1, Target: (1, 1, 1, 49), Block: 1
  - Source: (130, 259, 49, 49), Type: 4
  - Source: (130, 1, 1, 1), Type: 2
  - Source: (1, 130, 49, 49), Type: 4
  - Source: (1, 1, 1, 49), Type: -1, Target: (10, 10, 1, 49), Block: 1
[Block 1] B shape: (10, 49, 1)
  - Source: (10, 1, 1, 1), Type: 2
  - Source: (10, 10, 1, 49), Type: -1, Target: (1, 1, 1, 49), Block: 0
  - Source: (1, 10, 49, 49), Type: 4
  - Source: (1, 1, 1, 49), Type: -1, Target: (259, 259, 1, 49), Block: 0
[Block 0]  shape: (259, 49, 1)
  - BC type 2, source (259, 1, 1, 1), target_block N/A, target N/A
  - BC type -1, source (259, 259, 1, 

# 计算网格几何参数

In [3]:
mesh_geocal = MeshGeoCalculator2D(mesh_read)
mesh_geocal.compute_centroids()
mesh_geocal.compute_volumes()
mesh_geocal.compute_face_vectors()
for i, item in enumerate(mesh_geocal.mesh.blocks):
    print(f"block {i} keys: {list(item.keys())}")
print(mesh_geocal.mesh.blocks[0]['bc'])

[Block 0] xc range: (-13.69490000, 14.69897015), yc range: (-14.45284450, 14.45210219)
[Block 1] xc range: (1.00027755, 14.70264261), yc range: (-0.12138114, 0.17834181)
[Block 0] volume range: (3.62171595e-07, 1.78960182e+00)
[Block 1] volume range: (3.85062192e-07, 1.38875202e-01)
block 0 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S4']
block 1 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S4']
[{'type': 2, 'source': (259, 1, 1, 1)}, {'source': (259, 259, 1, 49), 'type': -1, 'target': (1, 1, 1, 49), 'target_block': 1, 'transform': (1, 2)}, {'type': 4, 'source': (1, 259, 49, 49)}, {'source': (1, 1, 1, 49), 'type': -1, 'target': (10, 10, 1, 49), 'target_block': 1, 'transform': (1, 2)}]


# 添加虚网格

In [4]:
blocks = np.copy(mesh_geocal.mesh.blocks)
gc.crate_ghost_cells(blocks, config.GHOST_LAYER, config.N_C)
for block in blocks:
 for bc in block['bc']: 
    if 'ghost_cell' in bc:
        print("ghost_cell shape:", bc['ghost_cell'].shape)
for i, block in enumerate(blocks):
    print(f"Block {i} keys:", list(block.keys()))

ghost_cell shape: (258, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (258, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (9, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (9, 2, 4)
ghost_cell shape: (48, 2, 4)
Block 0 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S4']
Block 1 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S4']


# 整理为一个用于计算的列表

In [8]:
blocks_cal = tf.trans_list2numpy_2d(blocks, config.N_C)
for i, block in enumerate(blocks_cal):
    print(f"Block {i} keys:", list(block.keys()))
    print(f"Block {i} has {len(block['bc'])} boundary conditions.")
    for j, bc in enumerate(block['bc']):
        print(f"  BC {j} keys: {list(bc.keys())}")
        print(f"    → type: {bc.get('type', 'N/A')}")

Block 0 keys: ['geo', 'fluid', 'bc']
Block 0 has 4 boundary conditions.
  BC 0 keys: ['type', 'source', 'ghost_cell']
    → type: 2
  BC 1 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
  BC 2 keys: ['type', 'source', 'ghost_cell']
    → type: 4
  BC 3 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
Block 1 keys: ['geo', 'fluid', 'bc']
Block 1 has 4 boundary conditions.
  BC 0 keys: ['source', 'type', 'ghost_cell']
    → type: 2
  BC 1 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
  BC 2 keys: ['source', 'type', 'ghost_cell']
    → type: 4
  BC 3 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1


# 初始化流场和边界条件